# Load topology proposal and old positions

In [1]:
from simtk.openmm import app, unit
from simtk import openmm
import mdtraj as md
import numpy as np
import pickle
import os
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmforcefields.generators import SystemGenerator

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [4]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/113/"
with open("/data/chodera/zhangi/perses_benchmark/neq/14/113/113_complex.pickle", "rb") as f:
    htf = pickle.load(f)
    topology_proposal = htf._topology_proposal

In [5]:
protein_filename = "../../input/rbd_protonated.pdb"
ligand_input = "../../input/ace2_protonated.pdb"

In [6]:
# First thing to do is load the apo protein to mutate...
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms


ligand_pdbfile = open(ligand_input, 'r')
ligand_pdb = app.PDBFile(ligand_pdbfile)
ligand_pdbfile.close()
ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
    ligand_pdb.topology)
ligand_n_atoms = ligand_md_topology.n_atoms

# Now create a complex
complex_md_topology = protein_md_topology.join(ligand_md_topology)
complex_topology = complex_md_topology.to_openmm()
complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
complex_positions[:protein_n_atoms, :] = protein_positions
complex_positions[protein_n_atoms:, :] = ligand_positions

# Convert positions back to openmm vec3 objects
complex_positions_vec3 = []
for position in complex_positions:
    complex_positions_vec3.append(openmm.Vec3(*position.value_in_unit_system(unit.md_unit_system)))
complex_positions = unit.Quantity(value=complex_positions_vec3, unit=unit.nanometer)

In [7]:
# Now generate solvated top, pos, sys
temperature = 300 * unit.kelvin
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}                                                             
nonperiodic_forcefield_kwargs = None
small_molecule_forcefields = 'gaff-2.11'
molecules = []
system_generator = SystemGenerator(forcefields=forcefield_files,
                                        barostat=barostat,
                                        forcefield_kwargs=forcefield_kwargs,
                                        periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                        nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                        small_molecule_forcefield=small_molecule_forcefields,
                                        molecules=molecules,
                                        cache=None)



modeller = app.Modeller(complex_topology, complex_positions)

# Now we have to add missing atoms
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=0.9 * unit.nanometers, ionicStrength=0.15 * unit.molar)

solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
solvated_system = system_generator.create_system(solvated_topology)


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


In [8]:
app.PDBFile.writeFile(solvated_topology, solvated_positions, open("solvated_input.pdb", 'w'), keepIds=True)

In [7]:
app.PDBFile.writeFile(complex_topology, complex_positions, open("complex_input.pdb", 'w'), keepIds=True)


# Run mutation in pymol

In [2]:
# Generate PDB of new topology/positions using pymol
mutant_position = "501" # assume only allowed_mutations only has one mutation
mutant_residue = "TYR" # assume only allowed_mutations only has one mutation
mutant_pdb = "output.pdb"
if os.path.exists(mutant_pdb):
    os.system(f"rm {mutant_pdb}")
os.system(f"python mutate.py rbd_.pdb {mutant_pdb} 1/{mutant_position}/ {mutant_residue}")       


 ObjectMolecule: Read crystal symmetry information.
 CmdLoad: "" appended into object "solvated_input", state 2.
Undo has been disabled. Reason: Object is too large
PyMOL>refresh_wizard
 ExecutiveRMSPairs: RMSD =    0.034 (4 to 4 atoms)
Mutating TYR...
 Error: failed to open file "output.pdb"
 Error: Argument processing aborted due to exception (above).
Selected!
 Mutagenesis: 4 rotamers loaded.
 Rotamer 4/4, strain=5.96
 Rotamer 4/4, strain=5.96
 Rotamer 1/4, strain=59.80


0

In [ ]:
# Prep PDBs for tleap
_logger.info("Prepping PDBs for tleap")
new_positions = self._prep_for_tleap(debug_dir, old_topology, new_topology, current_positions, int(mutant_position), mutant_residue, is_complex)


In [10]:
pdb = app.PDBFile("manual4.pdb")

In [11]:
for res in pdb.topology.residues():
    if res.name == 'HOH':
        print(res)
        for bond in res.bonds():
            print(bond)

<Residue 975 (HOH) of chain 7>
Bond(<Atom 15975 (H1) of chain 7 residue 975 (HOH)>, <Atom 15974 (O) of chain 7 residue 975 (HOH)>)
Bond(<Atom 15976 (H2) of chain 7 residue 975 (HOH)>, <Atom 15974 (O) of chain 7 residue 975 (HOH)>)
<Residue 976 (HOH) of chain 7>
Bond(<Atom 15978 (H1) of chain 7 residue 976 (HOH)>, <Atom 15977 (O) of chain 7 residue 976 (HOH)>)
Bond(<Atom 15979 (H2) of chain 7 residue 976 (HOH)>, <Atom 15977 (O) of chain 7 residue 976 (HOH)>)
<Residue 977 (HOH) of chain 7>
Bond(<Atom 15981 (H1) of chain 7 residue 977 (HOH)>, <Atom 15980 (O) of chain 7 residue 977 (HOH)>)
Bond(<Atom 15982 (H2) of chain 7 residue 977 (HOH)>, <Atom 15980 (O) of chain 7 residue 977 (HOH)>)
<Residue 978 (HOH) of chain 7>
Bond(<Atom 15984 (H1) of chain 7 residue 978 (HOH)>, <Atom 15983 (O) of chain 7 residue 978 (HOH)>)
Bond(<Atom 15985 (H2) of chain 7 residue 978 (HOH)>, <Atom 15983 (O) of chain 7 residue 978 (HOH)>)
<Residue 979 (HOH) of chain 7>
Bond(<Atom 15987 (H1) of chain 7 residue 979 

<Residue 1011 (HOH) of chain 7>
Bond(<Atom 16083 (H1) of chain 7 residue 1011 (HOH)>, <Atom 16082 (O) of chain 7 residue 1011 (HOH)>)
Bond(<Atom 16084 (H2) of chain 7 residue 1011 (HOH)>, <Atom 16082 (O) of chain 7 residue 1011 (HOH)>)
<Residue 1012 (HOH) of chain 7>
Bond(<Atom 16086 (H1) of chain 7 residue 1012 (HOH)>, <Atom 16085 (O) of chain 7 residue 1012 (HOH)>)
Bond(<Atom 16087 (H2) of chain 7 residue 1012 (HOH)>, <Atom 16085 (O) of chain 7 residue 1012 (HOH)>)
<Residue 1013 (HOH) of chain 7>
Bond(<Atom 16089 (H1) of chain 7 residue 1013 (HOH)>, <Atom 16088 (O) of chain 7 residue 1013 (HOH)>)
Bond(<Atom 16090 (H2) of chain 7 residue 1013 (HOH)>, <Atom 16088 (O) of chain 7 residue 1013 (HOH)>)
<Residue 1014 (HOH) of chain 7>
Bond(<Atom 16092 (H1) of chain 7 residue 1014 (HOH)>, <Atom 16091 (O) of chain 7 residue 1014 (HOH)>)
Bond(<Atom 16093 (H2) of chain 7 residue 1014 (HOH)>, <Atom 16091 (O) of chain 7 residue 1014 (HOH)>)
<Residue 1015 (HOH) of chain 7>
Bond(<Atom 16095 (H1) of

<Residue 1047 (HOH) of chain 7>
Bond(<Atom 16191 (H1) of chain 7 residue 1047 (HOH)>, <Atom 16190 (O) of chain 7 residue 1047 (HOH)>)
Bond(<Atom 16192 (H2) of chain 7 residue 1047 (HOH)>, <Atom 16190 (O) of chain 7 residue 1047 (HOH)>)
<Residue 1048 (HOH) of chain 7>
Bond(<Atom 16194 (H1) of chain 7 residue 1048 (HOH)>, <Atom 16193 (O) of chain 7 residue 1048 (HOH)>)
Bond(<Atom 16195 (H2) of chain 7 residue 1048 (HOH)>, <Atom 16193 (O) of chain 7 residue 1048 (HOH)>)
<Residue 1049 (HOH) of chain 7>
Bond(<Atom 16197 (H1) of chain 7 residue 1049 (HOH)>, <Atom 16196 (O) of chain 7 residue 1049 (HOH)>)
Bond(<Atom 16198 (H2) of chain 7 residue 1049 (HOH)>, <Atom 16196 (O) of chain 7 residue 1049 (HOH)>)
<Residue 1050 (HOH) of chain 7>
Bond(<Atom 16200 (H1) of chain 7 residue 1050 (HOH)>, <Atom 16199 (O) of chain 7 residue 1050 (HOH)>)
Bond(<Atom 16201 (H2) of chain 7 residue 1050 (HOH)>, <Atom 16199 (O) of chain 7 residue 1050 (HOH)>)
<Residue 1051 (HOH) of chain 7>
Bond(<Atom 16203 (H1) of

<Residue 1083 (HOH) of chain 7>
Bond(<Atom 16299 (H1) of chain 7 residue 1083 (HOH)>, <Atom 16298 (O) of chain 7 residue 1083 (HOH)>)
Bond(<Atom 16300 (H2) of chain 7 residue 1083 (HOH)>, <Atom 16298 (O) of chain 7 residue 1083 (HOH)>)
<Residue 1084 (HOH) of chain 7>
Bond(<Atom 16302 (H1) of chain 7 residue 1084 (HOH)>, <Atom 16301 (O) of chain 7 residue 1084 (HOH)>)
Bond(<Atom 16303 (H2) of chain 7 residue 1084 (HOH)>, <Atom 16301 (O) of chain 7 residue 1084 (HOH)>)
<Residue 1085 (HOH) of chain 7>
Bond(<Atom 16305 (H1) of chain 7 residue 1085 (HOH)>, <Atom 16304 (O) of chain 7 residue 1085 (HOH)>)
Bond(<Atom 16306 (H2) of chain 7 residue 1085 (HOH)>, <Atom 16304 (O) of chain 7 residue 1085 (HOH)>)
<Residue 1086 (HOH) of chain 7>
Bond(<Atom 16308 (H1) of chain 7 residue 1086 (HOH)>, <Atom 16307 (O) of chain 7 residue 1086 (HOH)>)
Bond(<Atom 16309 (H2) of chain 7 residue 1086 (HOH)>, <Atom 16307 (O) of chain 7 residue 1086 (HOH)>)
<Residue 1087 (HOH) of chain 7>
Bond(<Atom 16311 (H1) of

<Residue 1119 (HOH) of chain 7>
Bond(<Atom 16407 (H1) of chain 7 residue 1119 (HOH)>, <Atom 16406 (O) of chain 7 residue 1119 (HOH)>)
Bond(<Atom 16408 (H2) of chain 7 residue 1119 (HOH)>, <Atom 16406 (O) of chain 7 residue 1119 (HOH)>)
<Residue 1120 (HOH) of chain 7>
Bond(<Atom 16410 (H1) of chain 7 residue 1120 (HOH)>, <Atom 16409 (O) of chain 7 residue 1120 (HOH)>)
Bond(<Atom 16411 (H2) of chain 7 residue 1120 (HOH)>, <Atom 16409 (O) of chain 7 residue 1120 (HOH)>)
<Residue 1121 (HOH) of chain 7>
Bond(<Atom 16413 (H1) of chain 7 residue 1121 (HOH)>, <Atom 16412 (O) of chain 7 residue 1121 (HOH)>)
Bond(<Atom 16414 (H2) of chain 7 residue 1121 (HOH)>, <Atom 16412 (O) of chain 7 residue 1121 (HOH)>)
<Residue 1122 (HOH) of chain 7>
Bond(<Atom 16416 (H1) of chain 7 residue 1122 (HOH)>, <Atom 16415 (O) of chain 7 residue 1122 (HOH)>)
Bond(<Atom 16417 (H2) of chain 7 residue 1122 (HOH)>, <Atom 16415 (O) of chain 7 residue 1122 (HOH)>)
<Residue 1123 (HOH) of chain 7>
Bond(<Atom 16419 (H1) of

...

<Residue 3601 (HOH) of chain 7>
Bond(<Atom 23853 (H1) of chain 7 residue 3601 (HOH)>, <Atom 23852 (O) of chain 7 residue 3601 (HOH)>)
Bond(<Atom 23854 (H2) of chain 7 residue 3601 (HOH)>, <Atom 23852 (O) of chain 7 residue 3601 (HOH)>)
<Residue 3602 (HOH) of chain 7>
Bond(<Atom 23856 (H1) of chain 7 residue 3602 (HOH)>, <Atom 23855 (O) of chain 7 residue 3602 (HOH)>)
Bond(<Atom 23857 (H2) of chain 7 residue 3602 (HOH)>, <Atom 23855 (O) of chain 7 residue 3602 (HOH)>)
<Residue 3603 (HOH) of chain 7>
Bond(<Atom 23859 (H1) of chain 7 residue 3603 (HOH)>, <Atom 23858 (O) of chain 7 residue 3603 (HOH)>)
Bond(<Atom 23860 (H2) of chain 7 residue 3603 (HOH)>, <Atom 23858 (O) of chain 7 residue 3603 (HOH)>)
<Residue 3604 (HOH) of chain 7>
Bond(<Atom 23862 (H1) of chain 7 residue 3604 (HOH)>, <Atom 23861 (O) of chain 7 residue 3604 (HOH)>)
Bond(<Atom 23863 (H2) of chain 7 residue 3604 (HOH)>, <Atom 23861 (O) of chain 7 residue 3604 (HOH)>)
<Residue 3605 (HOH) of chain 7>
Bond(<Atom 23865 (H1) of

<Residue 3637 (HOH) of chain 7>
Bond(<Atom 23961 (H1) of chain 7 residue 3637 (HOH)>, <Atom 23960 (O) of chain 7 residue 3637 (HOH)>)
Bond(<Atom 23962 (H2) of chain 7 residue 3637 (HOH)>, <Atom 23960 (O) of chain 7 residue 3637 (HOH)>)
<Residue 3638 (HOH) of chain 7>
Bond(<Atom 23964 (H1) of chain 7 residue 3638 (HOH)>, <Atom 23963 (O) of chain 7 residue 3638 (HOH)>)
Bond(<Atom 23965 (H2) of chain 7 residue 3638 (HOH)>, <Atom 23963 (O) of chain 7 residue 3638 (HOH)>)
<Residue 3639 (HOH) of chain 7>
Bond(<Atom 23967 (H1) of chain 7 residue 3639 (HOH)>, <Atom 23966 (O) of chain 7 residue 3639 (HOH)>)
Bond(<Atom 23968 (H2) of chain 7 residue 3639 (HOH)>, <Atom 23966 (O) of chain 7 residue 3639 (HOH)>)
<Residue 3640 (HOH) of chain 7>
Bond(<Atom 23970 (H1) of chain 7 residue 3640 (HOH)>, <Atom 23969 (O) of chain 7 residue 3640 (HOH)>)
Bond(<Atom 23971 (H2) of chain 7 residue 3640 (HOH)>, <Atom 23969 (O) of chain 7 residue 3640 (HOH)>)
<Residue 3641 (HOH) of chain 7>
Bond(<Atom 23973 (H1) of

<Residue 3673 (HOH) of chain 7>
Bond(<Atom 24069 (H1) of chain 7 residue 3673 (HOH)>, <Atom 24068 (O) of chain 7 residue 3673 (HOH)>)
Bond(<Atom 24070 (H2) of chain 7 residue 3673 (HOH)>, <Atom 24068 (O) of chain 7 residue 3673 (HOH)>)
<Residue 3674 (HOH) of chain 7>
Bond(<Atom 24072 (H1) of chain 7 residue 3674 (HOH)>, <Atom 24071 (O) of chain 7 residue 3674 (HOH)>)
Bond(<Atom 24073 (H2) of chain 7 residue 3674 (HOH)>, <Atom 24071 (O) of chain 7 residue 3674 (HOH)>)
<Residue 3675 (HOH) of chain 7>
Bond(<Atom 24075 (H1) of chain 7 residue 3675 (HOH)>, <Atom 24074 (O) of chain 7 residue 3675 (HOH)>)
Bond(<Atom 24076 (H2) of chain 7 residue 3675 (HOH)>, <Atom 24074 (O) of chain 7 residue 3675 (HOH)>)
<Residue 3676 (HOH) of chain 7>
Bond(<Atom 24078 (H1) of chain 7 residue 3676 (HOH)>, <Atom 24077 (O) of chain 7 residue 3676 (HOH)>)
Bond(<Atom 24079 (H2) of chain 7 residue 3676 (HOH)>, <Atom 24077 (O) of chain 7 residue 3676 (HOH)>)
<Residue 3677 (HOH) of chain 7>
Bond(<Atom 24081 (H1) of

<Residue 3709 (HOH) of chain 7>
Bond(<Atom 24177 (H1) of chain 7 residue 3709 (HOH)>, <Atom 24176 (O) of chain 7 residue 3709 (HOH)>)
Bond(<Atom 24178 (H2) of chain 7 residue 3709 (HOH)>, <Atom 24176 (O) of chain 7 residue 3709 (HOH)>)
<Residue 3710 (HOH) of chain 7>
Bond(<Atom 24180 (H1) of chain 7 residue 3710 (HOH)>, <Atom 24179 (O) of chain 7 residue 3710 (HOH)>)
Bond(<Atom 24181 (H2) of chain 7 residue 3710 (HOH)>, <Atom 24179 (O) of chain 7 residue 3710 (HOH)>)
<Residue 3711 (HOH) of chain 7>
Bond(<Atom 24183 (H1) of chain 7 residue 3711 (HOH)>, <Atom 24182 (O) of chain 7 residue 3711 (HOH)>)
Bond(<Atom 24184 (H2) of chain 7 residue 3711 (HOH)>, <Atom 24182 (O) of chain 7 residue 3711 (HOH)>)
<Residue 3712 (HOH) of chain 7>
Bond(<Atom 24186 (H1) of chain 7 residue 3712 (HOH)>, <Atom 24185 (O) of chain 7 residue 3712 (HOH)>)
Bond(<Atom 24187 (H2) of chain 7 residue 3712 (HOH)>, <Atom 24185 (O) of chain 7 residue 3712 (HOH)>)
<Residue 3713 (HOH) of chain 7>
Bond(<Atom 24189 (H1) of

KeyboardInterrupt: 